In [1]:
import pandas as pd

# 1. Cargar datos
path = '/home/diego/proyectos/betavae-xai-ad/data/SubjectsData_AAL3_procesado2.csv'
df = pd.read_csv(path)

# 2. Preprocesamiento para asegurar consistencia con "Table 1" clínica
# a) Asegurar que cada SubjectID sea único (si el CSV tiene múltiples visitas, nos quedamos con la primera)
df_unique = df.drop_duplicates(subset='SubjectID', keep='first').copy()

# b) Estandarizar Grupos: A veces ADNI usa 'EMCI'/'LMCI'. La tabla pide 'MCI' unificado.
#    Si tu columna ya tiene solo 'MCI', esto no afectará.
df_unique['Group_Table'] = df_unique['ResearchGroup'].replace({
    'EMCI': 'MCI', 
    'LMCI': 'MCI', 
    'Early MCI': 'MCI', 
    'Late MCI': 'MCI'
})

# c) Filtrar solo los grupos de interés (ignorar 'SMC' o 'Patient' si existen y no están en el paper)
target_groups = ['AD', 'CN', 'MCI']
df_final = df_unique[df_unique['Group_Table'].isin(target_groups)].copy()

# 3. Función auxiliar para calcular la fila de la tabla
def calculate_demographics(subset):
    # N
    count = len(subset)
    
    # Age (Media ± SD)
    age_mean = subset['Age'].mean()
    age_sd = subset['Age'].std()
    age_str = f"{age_mean:.1f} ± {age_sd:.1f}"
    
    # Sex (M/F)
    # Asumiendo 'M' y 'F' (o normalizamos a mayúsculas por si acaso)
    sex_counts = subset['Sex'].str.upper().value_counts()
    males = sex_counts.get('M', 0)
    females = sex_counts.get('F', 0)
    sex_str = f"{males}/{females}"
    
    return [count, age_str, sex_str]

# 4. Generar la Tabla
data = {}

# Columnas por grupo
for group in target_groups:
    data[group] = calculate_demographics(df_final[df_final['Group_Table'] == group])

# Columna Total
data['Total'] = calculate_demographics(df_final)

# 5. Crear DataFrame de visualización
indices = ['Number', 'Age (years, mean ± SD)', 'Sex (Male/Female)']
table_check = pd.DataFrame(data, index=indices)

print("=== TABLA GENERADA DESDE EL CSV ===")
display(table_check)

print("\n=== VERIFICACIÓN DE N TOTAL ===")
print(f"Filas originales en CSV: {len(df)}")
print(f"Sujetos únicos (SubjectID): {len(df_unique)}")
print(f"Sujetos en grupos AD/CN/MCI: {len(df_final)}")

=== TABLA GENERADA DESDE EL CSV ===


,AD,CN,MCI,Total
Number,95,89,250,434
"Age (years, mean ± SD)",74.3 ± 7.5,74.4 ± 7.4,73.4 ± 8.0,73.8 ± 7.7
Sex (Male/Female),55/40,37/52,135/115,227/207



=== VERIFICACIÓN DE N TOTAL ===
Filas originales en CSV: 434
Sujetos únicos (SubjectID): 434
Sujetos en grupos AD/CN/MCI: 434


In [4]:
import numpy as np
import pandas as pd

# Cargar datos
csv_path = '/home/diego/proyectos/betavae-xai-ad/data/SubjectsData_AAL3_procesado2.csv'
npz_path = '/home/diego/Escritorio/limpio/AAL3_dynamicROIs_fmri_tensor_NeuroEnhanced_v6.5.17_AAL3_131ROIs_OMST_GCE_Signed_GrangerLag1_ChNorm_ROIreorderedYeo17_ParallelTuned/GLOBAL_TENSOR_from_AAL3_dynamicROIs_fmri_tensor_NeuroEnhanced_v6.5.17_AAL3_131ROIs_OMST_GCE_Signed_GrangerLag1_ChNorm_ROIreorderedYeo17_ParallelTuned.npz'

# Leer IDs del CSV (Cohorte Total)
df = pd.read_csv(csv_path)
ids_csv = set(df['SubjectID'].astype(str).str.strip())

# Leer IDs del Tensor (Cohorte Analizada)
data_npz = np.load(npz_path)
ids_tensor = set(data_npz['subject_ids'].astype(str))

# Encontrar la diferencia
excluded_subjects = ids_csv - ids_tensor

print(f"Sujetos en CSV: {len(ids_csv)}")
print(f"Sujetos en Tensor: {len(ids_tensor)}")
print(f"Sujetos Excluidos ({len(excluded_subjects)}): {excluded_subjects}")
print("Estos sujetos fueron excluidos del análisis porque no estaban presentes en el tensor de datos funcionales.")
# a que clase pertenecen esos sujetos excluidos?
excluded_info = df[df['SubjectID'].astype(str).str.strip().isin(excluded_subjects)][['SubjectID', 'ResearchGroup']]
print("\nInformación de los sujetos excluidos:")
display(excluded_info)


Sujetos en CSV: 434
Sujetos en Tensor: 431
Sujetos Excluidos (3): {'127_S_4301', '127_S_4765', '035_S_7021'}
Estos sujetos fueron excluidos del análisis porque no estaban presentes en el tensor de datos funcionales.

Información de los sujetos excluidos:


,SubjectID,ResearchGroup
201,035_S_7021,MCI
298,127_S_4301,EMCI
299,127_S_4765,EMCI


In [5]:
import pandas as pd
import numpy as np

# 1. Cargar datos originales
path = '/home/diego/proyectos/betavae-xai-ad/data/SubjectsData_AAL3_procesado2.csv'
df = pd.read_csv(path)

# 2. Definir explícitamente los sujetos a excluir (que no estaban en el tensor)
excluded_ids = {'127_S_4301', '127_S_4765', '035_S_7021'}

# 3. Filtrar el DataFrame (Limpieza)
# Normalizamos strings por si acaso hay espacios
df['SubjectID'] = df['SubjectID'].astype(str).str.strip()
df_clean = df[~df['SubjectID'].isin(excluded_ids)].copy()

# 4. Estandarizar Grupos (MCI, EMCI, LMCI -> MCI)
df_clean['Group_Table'] = df_clean['ResearchGroup'].replace({
    'EMCI': 'MCI', 
    'LMCI': 'MCI', 
    'Early MCI': 'MCI', 
    'Late MCI': 'MCI'
})

# Filtrar solo los grupos de interés (por si hay SMC u otros)
target_groups = ['AD', 'CN', 'MCI']
df_clean = df_clean[df_clean['Group_Table'].isin(target_groups)].copy()

# 5. Función de cálculo de estadísticas
def get_stats(subset):
    if len(subset) == 0: return ["0", "0.0 ± 0.0", "0/0"]
    
    # N
    n = len(subset)
    
    # Age
    age_mean = subset['Age'].mean()
    age_sd = subset['Age'].std()
    age_str = f"{age_mean:.1f} $\\pm$ {age_sd:.1f}"
    
    # Sex
    sex_counts = subset['Sex'].str.upper().value_counts()
    males = sex_counts.get('M', 0)
    females = sex_counts.get('F', 0)
    sex_str = f"{males}/{females}"
    
    return n, age_str, sex_str

# 6. Calcular datos para cada columna
stats = {}
for g in target_groups:
    stats[g] = get_stats(df_clean[df_clean['Group_Table'] == g])

stats['Total'] = get_stats(df_clean)

# 7. Imprimir resultados en formato legible y Generar LaTeX
print(f"=== DATOS FINALES (N={len(df_clean)}) ===")
print(f"{'Metric':<10} | {'AD':<15} | {'CN':<15} | {'MCI':<15} | {'Total':<15}")
print("-" * 80)
print(f"{'Number':<10} | {stats['AD'][0]:<15} | {stats['CN'][0]:<15} | {stats['MCI'][0]:<15} | {stats['Total'][0]:<15}")
print(f"{'Age':<10} | {stats['AD'][1]:<15} | {stats['CN'][1]:<15} | {stats['MCI'][1]:<15} | {stats['Total'][1]:<15}")
print(f"{'Sex':<10} | {stats['AD'][2]:<15} | {stats['CN'][2]:<15} | {stats['MCI'][2]:<15} | {stats['Total'][2]:<15}")

print("\n" + "="*30 + " CÓDIGO LATEX " + "="*30)
latex_code = f"""
\\begin{{table}}[h!]
\\centering
\\caption{{Demographic and clinical characteristics of the study cohort.}}
\\label{{tab:cohort}}
\\begin{{tabular}}{{lcccc}}
\\hline
& AD & CN & MCI & Total \\\\
\\hline
Number & {stats['AD'][0]} & {stats['CN'][0]} & {stats['MCI'][0]} & {stats['Total'][0]} \\\\
Age (years, mean $\\pm$ SD) & {stats['AD'][1]} & {stats['CN'][1]} & {stats['MCI'][1]} & {stats['Total'][1]} \\\\
Sex (Male/Female) & {stats['AD'][2]} & {stats['CN'][2]} & {stats['MCI'][2]} & {stats['Total'][2]} \\\\
\\hline
\\end{{tabular}}
\\end{{table}}
"""
print(latex_code)
print("="*74)

=== DATOS FINALES (N=431) ===
Metric     | AD              | CN              | MCI             | Total          
--------------------------------------------------------------------------------
Number     | 95              | 89              | 247             | 431            
Age        | 74.3 $\pm$ 7.5  | 74.4 $\pm$ 7.4  | 73.3 $\pm$ 8.0  | 73.8 $\pm$ 7.7 
Sex        | 55/40           | 37/52           | 133/114         | 225/206        

============================== CÓDIGO LATEX ==============================

\begin{table}[h!]
\centering
\caption{Demographic and clinical characteristics of the study cohort.}
\label{tab:cohort}
\begin{tabular}{lcccc}
\hline
& AD & CN & MCI & Total \\
\hline
Number & 95 & 89 & 247 & 431 \\
Age (years, mean $\pm$ SD) & 74.3 $\pm$ 7.5 & 74.4 $\pm$ 7.4 & 73.3 $\pm$ 8.0 & 73.8 $\pm$ 7.7 \\
Sex (Male/Female) & 55/40 & 37/52 & 133/114 & 225/206 \\
\hline
\end{tabular}
\end{table}



In [6]:
# =================== CELDA DE VERIFICACIÓN DE ESCÁNERES ===================

# 1. Función para normalizar los nombres de los fabricantes
def map_manufacturer(name):
    name = str(name).upper()
    if 'PHILIPS' in name:
        return 'Philips'
    elif 'SIEMENS' in name:
        return 'Siemens'
    elif 'GE' in name:
        return 'GE'
    else:
        return 'Other'

# 2. Calcular conteos sobre el DataFrame limpio (N=431)
# Asumimos que la columna se llama 'Manufacturer'. Si se llama distinto, ajusta el string.
if 'Manufacturer' in df_clean.columns:
    scanner_counts = df_clean['Manufacturer'].apply(map_manufacturer).value_counts()
    
    n_philips = scanner_counts.get('Philips', 0)
    n_siemens = scanner_counts.get('Siemens', 0)
    n_ge = scanner_counts.get('GE', 0)
    total_scanners = scanner_counts.sum()

    # 3. Imprimir reporte
    print(f"=== DISTRIBUCIÓN DE ESCÁNERES (N={total_scanners}) ===")
    print(f"Original en CSV: {df_clean['Manufacturer'].unique()}")
    print("-" * 50)
    print(f"Philips: {n_philips}")
    print(f"Siemens: {n_siemens}")
    print(f"GE:      {n_ge}")
    print("-" * 50)

    # 4. Generar la frase para el paper
    print("\n>>> FRASE SUGERIDA PARA EL PAPER:")
    sentence = f"Of the {total_scanners} images, {n_philips} were acquired using Philips, {n_siemens} using Siemens, and {n_ge} using GE scanners."
    print(sentence)
    
    # 5. Chequeo rápido con tus números actuales
    matches = (n_philips == 272) and (n_siemens == 111) and (n_ge == 48)
    if matches:
        print("\n✅ ¡Los números COINCIDEN perfectamente con tu texto actual!")
    else:
        print("\n⚠️ ATENCIÓN: Los números han cambiado al filtrar los 3 sujetos. Actualiza el texto.")

else:
    print("❌ No se encontró la columna 'Manufacturer' en el DataFrame.")
    print("Columnas disponibles:", df_clean.columns.tolist())

=== DISTRIBUCIÓN DE ESCÁNERES (N=431) ===
Original en CSV: ['Philips' 'SIEMENS' 'GE MEDICAL SYSTEMS']
--------------------------------------------------
Philips: 274
Siemens: 110
GE:      47
--------------------------------------------------

>>> FRASE SUGERIDA PARA EL PAPER:
Of the 431 images, 274 were acquired using Philips, 110 using Siemens, and 47 using GE scanners.

⚠️ ATENCIÓN: Los números han cambiado al filtrar los 3 sujetos. Actualiza el texto.
